# 2020_10_03: String of words approach

Here, we instead of having the embedding for the whole sentence, we use an embedding for each word, and we pad the rest of it to zero.

In [1]:
DSET_FOLDER_PATH = './dataset/quora/'
GLOVE_FOLDER_PATH = './embeddings/glove/'
CORPUS_FOLDER_PATH = './corpi/'
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt  
import wordcloud as wc 
import seaborn as sns 
import nltk
import re
import contractions
import symspellpy
train_dset_df = pd.read_csv(DSET_FOLDER_PATH + "train.csv")
from tqdm import tqdm 
tqdm.pandas()
import gc

In [2]:
import utils.PreprocessingEmbedding20201003 as pped

In [3]:
preprocessor = pped.Preprocessor(CORPUS_FOLDER_PATH + "words_alpha.txt")

In [4]:
train_dset_df["preprocessed"] = train_dset_df.question_text.progress_apply(lambda x: preprocessor.preprocess(x, stem_reduce=False))

100%|██████████| 783673/783673 [02:49&lt;00:00, 4631.08it/s]


In [5]:
train_dset_df["preprocessed"] = train_dset_df["preprocessed"].progress_apply(lambda x: x[:min(len(x), 20)])

100%|██████████| 783673/783673 [00:01&lt;00:00, 482975.59it/s]


In [6]:
preprocessor = None 
gc.collect()
embedder = pped.Glove_Embedder(GLOVE_FOLDER_PATH+"glove.6B.50d.txt")

In [7]:
def embed_sentence_as_twenty(sentence, embedder, embedding_size):
    '''
    sentence: A list of strs. The preprocessed sentence.
    embedder: A pped.Glove_Embedder (2020_10_03 compliant)
    embedding_size: The size of the embedding vector (50, 100 or 200)
    '''
    sentence_embedding = []
    for word in sentence:
        word_embedding = embedder.get_embedding_for_word(word)
        sentence_embedding += word_embedding
    sentence_embedding += [0]*(embedding_size*20 - len(sentence_embedding))
    return sentence_embedding



In [8]:
train_dset_df["embedded"] = train_dset_df.question_text.progress_apply(lambda x: embed_sentence_as_twenty(x, embedder, 50))

  0%|          | 1/783673 [00:00&lt;3:56:26, 55.24it/s]


NameError: name &#39;embed_sentence_as_ten&#39; is not defined

### See 2020_10_03_string_of_words_part_2.ipynb

In [ ]:
X = np.array(train_dset_df["embedded"].to_list())
y = train_dset_df["target"].to_numpy()

In [ ]:
import sklearn.svm 
svm = sklearn.svm.LinearSVC(max_iter=10000)
svm.fit(X, y)